# RealAgents Housing Price Prediction 
## Author : Owen Nda Diche
This notebook recreates a five-part data science workflow for a fictional real estate company (RealAgents):

1. **Task 1:** Count missing values in the `city` column  
2. **Task 2:** Build a cleaned dataset (`clean_data`) following strict rules (missing value handling, string cleaning, data types)  
3. **Task 3:** Aggregate average and variance of sale price by number of bedrooms (`price_by_rooms`)  
4. **Task 4:** Train a **baseline** model and generate predictions (`base_result`)  
5. **Task 5:** Train a **comparison** model and generate predictions (`compare_result`)

**Dataset:** `Housing.csv`


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error

# Load the data
df = pd.read_csv("Housing.csv")
df.head()

,index,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8 sq.m.
1,1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8 sq.m.
2,2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5 sq.m.
3,3,1773666,Silvertown,373251,2020-04-13,6.1,6,Det.,528.4 sq.m.
4,4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1 sq.m.


## Task 1 — Count missing `city` values

In this dataset, missing `city` values are represented with `"--"` (two dashes).
We create an integer object called **`missing_city`** that stores the number of missing entries.


In [2]:
missing_city = int((df["city"].astype("string").str.strip() == "--").sum())
missing_city

73

## Task 2 — Create a cleaned dataframe (`clean_data`)

We clean the dataset using the following rules:

- **house_id:** nominal, no missing expected (strip whitespace)
- **city:** fill missing with `"Unknown"` and standardize text
- **sale_price:** convert to numeric; **remove rows** where missing; whole dollars (int)
- **sale_date:** fill missing with `"2023-01-01"` and standardize to `YYYY-MM-DD`
- **months_listed:** numeric; fill missing with mean; round to 1 decimal
- **bedrooms:** numeric; fill missing with mean; round to nearest integer
- **house_type:** standardize to `Terraced`, `Semi-detached`, `Detached`; fill missing with most common
- **area:** extract the first numeric value from strings like `"107.8 sq.m."`; fill missing with mean; round to 1 decimal

The final output must be a dataframe named **`clean_data`** with the columns:

`house_id, city, sale_price, sale_date, months_listed, bedrooms, house_type, area`


In [3]:
clean_data = df.copy()

# helpers
MISSING_MARKERS = {"--", "-", "---", "—", "–", "na", "n/a", "null", "none", "nan", "missing", ""}

def normalize_text(series: pd.Series) -> pd.Series:
    """Trim strings, collapse spaces, and convert common missing markers to <NA>."""
    s = series.astype("string")
    s = s.str.strip().str.replace(r"\s+", " ", regex=True)
    s_lower = s.str.lower()
    return s.mask(s_lower.isin(MISSING_MARKERS), pd.NA)

# house_id
clean_data["house_id"] = clean_data["house_id"].astype("string").str.strip()

# city
clean_data["city"] = normalize_text(clean_data["city"]).str.title().fillna("Unknown").astype("string")

# sale_price (drop missing)
clean_data["sale_price"] = pd.to_numeric(normalize_text(clean_data["sale_price"]), errors="coerce")
clean_data = clean_data.dropna(subset=["sale_price"])
clean_data["sale_price"] = clean_data["sale_price"].round(0).astype(int)

# sale_date (fill missing with 2023-01-01; keep ISO string)
clean_data["sale_date"] = normalize_text(clean_data["sale_date"]).fillna("2023-01-01")
clean_data["sale_date"] = pd.to_datetime(clean_data["sale_date"], errors="coerce").fillna(pd.Timestamp("2023-01-01"))
clean_data["sale_date"] = clean_data["sale_date"].dt.strftime("%Y-%m-%d").astype("string")

# months_listed (mean fill; 1 dp)
clean_data["months_listed"] = pd.to_numeric(normalize_text(clean_data["months_listed"]), errors="coerce")
ml_mean = clean_data["months_listed"].mean()
clean_data["months_listed"] = clean_data["months_listed"].fillna(ml_mean).round(1)

# bedrooms (mean fill; nearest int)
clean_data["bedrooms"] = pd.to_numeric(normalize_text(clean_data["bedrooms"]), errors="coerce")
br_mean = clean_data["bedrooms"].mean()
clean_data["bedrooms"] = clean_data["bedrooms"].fillna(br_mean).round(0).astype(int)

# house_type (standardize; mode fill)
ht_raw = normalize_text(clean_data["house_type"]).astype("string").str.strip().str.lower()
ht_raw = ht_raw.str.replace(r"\.+$", "", regex=True)  # "det." -> "det"
ht_raw = ht_raw.str.replace("semi detached", "semi-detached", regex=False)
ht_raw = ht_raw.str.replace("semidetached", "semi-detached", regex=False)

ht_map = {
    "detached": "Detached", "det": "Detached",
    "semi-detached": "Semi-detached", "semi": "Semi-detached",
    "terraced": "Terraced", "terr": "Terraced",
}
clean_data["house_type"] = ht_raw.map(ht_map)

mode_ht = clean_data["house_type"].mode(dropna=True)
fill_ht = mode_ht.iloc[0] if len(mode_ht) else "Terraced"
clean_data["house_type"] = clean_data["house_type"].fillna(fill_ht).astype("string")

# area (extract first number; mean fill; 1 dp)
area_txt = normalize_text(clean_data["area"]).astype("string").str.strip()
area_num = (
    area_txt
    .str.replace(",", "", regex=False)
    .str.extract(r"(\d+(?:\.\d+)?)", expand=False)
)
clean_data["area"] = pd.to_numeric(area_num, errors="coerce")
area_mean = clean_data["area"].mean()
clean_data["area"] = clean_data["area"].fillna(area_mean).round(1)

# final column order
clean_data = clean_data[
    ["house_id", "city", "sale_price", "sale_date", "months_listed", "bedrooms", "house_type", "area"]
]

clean_data.head()

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5
3,1773666,Silvertown,373251,2020-04-13,6.1,6,Detached,528.4
4,1258487,Silvertown,328885,2020-09-24,8.7,5,Detached,477.1


### Quick validation checks

We confirm:
- no missing values remain
- data types look correct
- `area` has realistic summary statistics


In [4]:
clean_data.isna().sum(), clean_data.dtypes, clean_data["area"].describe()

(house_id         0
 city             0
 sale_price       0
 sale_date        0
 months_listed    0
 bedrooms         0
 house_type       0
 area             0
 dtype: int64,
 house_id         string[python]
 city             string[python]
 sale_price                int64
 sale_date        string[python]
 months_listed           Float64
 bedrooms                  int64
 house_type       string[python]
 area                    Float64
 dtype: object,
 count        1500.0
 mean     352.123933
 std      146.083507
 min           100.3
 25%           228.4
 50%           350.7
 75%         478.025
 max           600.0
 Name: area, dtype: Float64)

## Task 3 — Price by bedrooms (`price_by_rooms`)

We aggregate the cleaned data to compute:

- `avg_price`: mean sale price per bedroom count
- `var_price`: variance of sale price per bedroom count

All values are rounded to **1 decimal place**.


In [5]:
price_by_rooms = (
    clean_data.groupby("bedrooms", as_index=False)
    .agg(
        avg_price=("sale_price", "mean"),
        var_price=("sale_price", "var")
    )
)

price_by_rooms["avg_price"] = price_by_rooms["avg_price"].round(1)
price_by_rooms["var_price"] = price_by_rooms["var_price"].round(1)

price_by_rooms = price_by_rooms.sort_values("bedrooms").reset_index(drop=True)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


## Tasks 4 & 5 — Modeling

For a portfolio version, we create a **train/validation split** from `clean_data`:

- **Task 4 (Baseline):** Ridge Regression with one-hot encoding  
- **Task 5 (Comparison):** Histogram Gradient Boosting Regressor

We output two dataframes:
- **`base_result`**: `house_id` and baseline predicted `price`
- **`compare_result`**: `house_id` and comparison predicted `price`

We also compute **RMSE** on the validation split to compare models.


In [6]:
model_data = clean_data.copy()

# Convert sale_date into usable numeric features for ML (year and month)
dt = pd.to_datetime(model_data["sale_date"], errors="coerce")
model_data["sale_year"] = dt.dt.year.astype(int)
model_data["sale_month"] = dt.dt.month.astype(int)

# Define target and features
target = "sale_price"
id_col = "house_id"

X = model_data.drop(columns=[target])
y = model_data[target].astype(float)

# Train/validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Columns for preprocessing
cat_cols = ["city", "house_type"]
num_cols = ["months_listed", "bedrooms", "area", "sale_year", "sale_month"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ],
    remainder="drop"
)

### Task 4 — Baseline model (Ridge Regression)

This is a simple linear model with regularization, trained on the training split.
We generate `base_result` predictions for the validation split.


In [10]:
baseline_model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", Ridge(alpha=1.0, random_state=0))
    ]
)

baseline_model.fit(X_train, y_train)
base_preds = baseline_model.predict(X_valid)
base_preds = np.clip(base_preds, 0, None)  # sale_price must be >= 0

base_result = pd.DataFrame({
    "house_id": X_valid[id_col].astype("string"),
    "price": base_preds
})

base_rmse = np.sqrt(mean_squared_error(y_valid, base_preds))
base_rmse, base_result.head()

(np.float64(21463.72634033792),
      house_id          price
 1116  1320704   27547.348917
 1368  1089831  365075.972520
 422   1639742  212288.639207
 413   1421799  256641.528376
 451   1378910   57452.812687)

### Task 5 — Comparison model (Gradient Boosting)

This model captures non-linear relationships and interactions.  
We generate `compare_result` predictions for the validation split and compute RMSE.


In [11]:
compare_model = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", HistGradientBoostingRegressor(
            loss="squared_error",
            learning_rate=0.06,
            max_depth=6,
            max_iter=600,
            min_samples_leaf=20,
            random_state=0
        ))
    ]
)

compare_model.fit(X_train, y_train)
compare_preds = compare_model.predict(X_valid)
compare_preds = np.clip(compare_preds, 0, None)

compare_result = pd.DataFrame({
    "house_id": X_valid[id_col].astype("string"),
    "price": compare_preds
})

compare_rmse = np.sqrt(mean_squared_error(y_valid, compare_preds))
compare_rmse, compare_result.head()

(np.float64(16598.38972502542),
      house_id          price
 1116  1320704   36436.600183
 1368  1089831  375436.000216
 422   1639742  213721.630727
 413   1421799  250862.748371
 451   1378910   58662.304915)

## Model comparison summary

We compare RMSE on the validation split. Lower is better.


In [12]:
pd.DataFrame({
    "model": ["Baseline (Ridge)", "Comparison (HistGB)"],
    "rmse": [base_rmse, compare_rmse]
}).sort_values("rmse")


,model,rmse
1,Comparison (HistGB),16598.389725
0,Baseline (Ridge),21463.726340


## Insights and results

### Data quality and preparation
- Missing values were not always stored as standard `NaN`. In particular, several fields used placeholder strings such as `"--"` and mixed-format text (e.g., `area` stored as `"107.8 sq.m."`).
- A consistent cleaning function was applied across fields to:
  - Standardize text (trim whitespace, collapse repeated spaces, normalize case)
  - Convert placeholder markers to missing values
  - Apply rule-based imputation (mean/mode/date default) and required rounding
  - Extract numeric values from text fields (e.g., `area` extracted as the first number from `"xx.x sq.m."`)

### Price patterns by bedrooms
Using `price_by_rooms`, the average sale price increases as bedroom count increases, supporting the team’s assumption that bedrooms strongly influence price. Variance is also substantial, suggesting that location (`city`), property size (`area`), and property type (`house_type`) contribute meaningful additional signal beyond bedrooms alone.

### Model performance (RMSE)
Two supervised models were trained and evaluated on a held-out validation split:

- **Baseline (Ridge Regression):** RMSE ≈ **21,464**
- **Comparison (Histogram Gradient Boosting):** RMSE ≈ **16,598**

The boosted model performs significantly better, indicating that the pricing relationship is not purely linear and benefits from non-linear interactions (e.g., the combined effects of bedrooms, area, city, and house type).

### Practical takeaway
A gradient-boosting model can provide more accurate pricing guidance than a linear baseline for setting competitive listing prices. This can reduce underpricing/overpricing risk and potentially shorten time-to-sale by aligning listings closer to market expectations.
